In [5]:
# made by Gibeom LEE, HI LAB
# csv파일 여러개를 일정한 간격을 두어 합쳐줍니다. (다른 열)

In [6]:
import os
import pandas as pd
import numpy as np
import glob

In [7]:
# 설정 ----------------------------
folder = r"C:\Users\user\Desktop\Gibeom\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\8. Gibeom touch\260219_Cap별 peak 하나 계형\0minimized_Cut_csv"

#file_name_str = "Pristine_4cm_1.83Hz Coated_4cm_1.74Hz Pristine_4cm_2.44Hz Coated_4cm_2.65Hz Pristine_4cm_Many Coated_4cm_3.98Hz"  # 폴더 내에 일부 파일만 변경하려면 파일명 기입 필요

gap_ratio = 0.2  # 각 파일 사이의 시간 간격 비율 ex) 20%


In [8]:
# 폴더 이름 추출
folder_name = os.path.basename(folder)
output_filename = f"Merge_{folder_name}.csv"
output_path = os.path.join(folder, output_filename)


# 병합 함수 -----------------------
def merge_data(folder, output_path, gap_ratio, file_name_str=None):
    # 파일 선택: 일부 지정 또는 전체 폴더 내 파일
    if file_name_str is not None:
        file_names = file_name_str.split()
        file_paths = [os.path.join(folder, f"{name}.csv") for name in file_names]
    else:
        file_paths = glob.glob(os.path.join(folder, "*.csv"))
        # 00Merge 파일 제외
        file_paths = [f for f in file_paths if "Merge" not in os.path.basename(f)]

    # 파일명 기준 오름차순 정렬
    file_paths.sort(key=lambda x: os.path.basename(x).lower())

    if not file_paths:
        print("❌ CSV 파일이 없습니다. 경로를 확인하세요.")
        return

    print(f"📂 총 {len(file_paths)}개의 파일을 병합합니다 (이름순 오름차순):\n")
    for f in file_paths:
        print(" -", os.path.basename(f))

    # 각 파일 읽기
    dataframes = [pd.read_csv(fp, header=None) for fp in file_paths]
    shifted_data = []
    current_time_offset = 0.0

    # 시간축 이동 병합
    for i, df in enumerate(dataframes):
        time_vals = df.iloc[:, 0].values
        voltage_vals = df.iloc[:, 1].values

        shifted_time = time_vals + current_time_offset
        shifted_df = pd.DataFrame({
            "Time": shifted_time,
            f"V{i+1}": voltage_vals
        })
        shifted_data.append(shifted_df)

        # 시간 간격 계산
        last_time = time_vals[-1]
        rounded_duration = round(last_time, 6)
        time_gap = rounded_duration * gap_ratio
        current_time_offset += rounded_duration + time_gap

    # 병합 (시간 기준 outer join)
    merged_df = shifted_data[0]
    for df in shifted_data[1:]:
        merged_df = pd.merge(merged_df, df, on="Time", how="outer")

    # 시간 정렬 및 저장
    merged_df = merged_df.sort_values("Time").reset_index(drop=True)
    merged_df.to_csv(output_path, index=False, header=False, float_format="%.6f")

    print(f"\n✅ 병합 완료! 저장 경로: {output_path}")


# 실행 ----------------------------
merge_data(folder, output_path, gap_ratio)


📂 총 13개의 파일을 병합합니다 (이름순 오름차순):

 - Cut_01._0nF_TENG.csv
 - Cut_02._1nF_TENG.csv
 - Cut_03._3.3nF_TENG.csv
 - Cut_04._4.7nF_TENG.csv
 - Cut_05._6.8nF_TENG.csv
 - Cut_06._8.2nF_TENG.csv
 - Cut_07._10nF_TENG.csv
 - Cut_08._22nF_TENG.csv
 - Cut_09._33nF_TENG.csv
 - Cut_10._47nF_TENG.csv
 - Cut_11._68nF_TENG.csv
 - Cut_12._82nF_TENG.csv
 - Cut_13._220nF_TENG.csv


KeyboardInterrupt: 